In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import speed_of_light, Boltzmann, Planck

In [5]:
def distribution(j0, j1, energy, temp):
    KT = k_boltzmann_wavenumber*temp
    N0 = (2*j0+1)
    N1 = (2*j1+1)
    Gj = N1/N0

    delE = abs(energy[j0]-energy[j1])

    return Gj*np.exp(-delE/KT)

k_boltzmann_wavenumber = Boltzmann/1.98630e-23
def boltzman_distribution(energyLevels, temp=5):
    KT = k_boltzmann_wavenumber*temp
    Nj = [(2*i+1)*np.exp(-energy/KT) for i, energy in enumerate(energyLevels)]
    Nj = np.array(Nj, dtype=np.float)
    Nj = Nj/Nj.sum()
    return Nj

In [6]:
def mhz_wavenumber(freq):
    return freq*1e6/(speed_of_light*100)

mhz_wn = np.vectorize(mhz_wavenumber)

In [7]:
mhz_wn(20)

array(0.00066713)

In [8]:
Energy_CO = np.array([117692.360, 118101.812, 235380.046])
Energy_CO_in_wn = mhz_wn(Energy_CO)
Energy_CO_in_wn

array([3.92579456, 3.93945241, 7.85143321])

In [9]:
boltzman_distribution(Energy_CO_in_wn, 5)

array([0.17843945, 0.53321876, 0.28834179])

In [10]:
q_10 = 4e-11
q_01 = q_10*distribution(0, 1, Energy_CO_in_wn, 5)
q_01

1.1952934570844706e-10

In [11]:
q_01*distribution(1, 0, Energy_CO_in_wn, 5)

3.9686845793026256e-11

In [12]:
def kinetic_simulation_on(t, N):

    CD0, CD1, CD2, CDHe, CDHe2 = N

    p = branching_ratio

    # CD: j=0
    attachmentRate0 = -Rate_k31_0*CD0 + Rate_kCID1*CDHe*p
    collisionalRate0 = (-Rate_q_01*CD0 + Rate_q_10*CD1 + Rate_q_20*CD2 - Rate_q_02*CD0) if includeCollision else 0
    spontaneousEmissionRate = A_10*CD1
    stimulatedRate = -Rate_B_01*CD0 + Rate_B_10*CD1


    dCD0_dt = attachmentRate0 + collisionalRate0 + spontaneousEmissionRate + stimulatedRate

    # CD: j=1
    attachmentRate1 = -Rate_k31_1*CD1 + Rate_kCID1*CDHe*(1-p)
    collisionalRate1 = (Rate_q_01*CD0 - Rate_q_10*CD1 - Rate_q_12*CD1 + Rate_q_21*CD2) if includeCollision else 0

    dCD1_dt = attachmentRate1 + collisionalRate1 - spontaneousEmissionRate - stimulatedRate

    # CD: j=2
    collisionalRate2 = (Rate_q_02*CD0 - Rate_q_20*CD2 + Rate_q_12*CD1 - Rate_q_21*CD2) if includeCollision else CD2
    dCD2_dt = collisionalRate2

    # CDHe:
    attachmentRate2 = -Rate_k32*CDHe + Rate_kCID2*CDHe2
    dCDHe_dt = -attachmentRate0 - attachmentRate1 + attachmentRate2

    # CDHe2
    dCDHe2_dt = -attachmentRate2

    return [dCD0_dt, dCD1_dt, dCD2_dt, dCDHe_dt, dCDHe2_dt]

In [13]:
totalLevel = 5
q_excitation = []

In [14]:
def get_level_qRate(totallevel=6, samelevelExistInFile=True):
    q_rate_deexcitation = []
    counter = 0
    for n in range(1, totallevel):
        for _temp in range(n+1):
            if _temp != n: 
                q_rate_deexcitation.append(f"{n}{_temp}")
                if not samelevelExistInFile: counter += 1
            if samelevelExistInFile: counter += 1
    
    return q_rate_deexcitation

In [15]:
totallevel = 4
get_level_qRate(totallevel)

['10', '20', '21', '30', '31', '32']

In [16]:
from functools import reduce

def collisionalRateDistribution(t, N):
    
    global fullRateEquation
    _N = [f"N{i}" for i in range(totallevel)]
    _N_He = [f"{nHe}{i}" for i in range(2)]
    
    _rateCollection = []
    _temp0 = f"B_{excitedLevel-1}{excitedLevel}"
    _temp1 = f"B_{excitedLevel}{excitedLevel-1}"
    
    # defined B rate from excited state 
    _B_rate = f" + {_temp0}*{_N[excitedLevel-1]} - {_temp1}*{_N[excitedLevel]} "
    
    # For loop for combining different rates processes
    for i in range(totallevel):
        
        # Collisional rate
        _rateEquationsCombined = []
        for j in range(totallevel):
            if i!= j: 
                
                key = f"{j}{i}"
                keyInverse = f"{i}{j}"
                
                _temp0 = f"q_{key}"
                _temp1 = f"q_{keyInverse}"
                
                _k0 = f" + {_temp0}*{nHe}*{_N[j]} - {_temp1}*{nHe}*{_N[i]}"
                
                _rateEquationsCombined.append(_k0)
        
        
        # END: Collisional rate for loop
        _k1 = ""
        
        # Einstein Coefficient A
        if i == 0:
            _temp0 = f"A_10"
            _k1 += f" + {_temp0}*{_N[1]}"
        
        if includeSpontaneousEmissionForAllLevels:
            if i == totallevel-1:
                _temp0 = f"A_{i}{i-1}"
                _k1 += f" - {_temp0}*{_N[i]}"

            if i>0 and i<totallevel-1:
                _temp0 = f"A_{i}{i-1}"
                _temp1 = f"A_{i+1}{i}"
                _k1 += f" - {_temp0}*{_N[i]} + {_temp1}*{_N[i+1]}"
                
        elif i == 1:
            _temp0 = f"A_10"
            _k1 += f" - {_temp0}*{_N[1]}"
            
        # END: Einstein Coefficient A
        
        
        # Einstein Coefficient B
        # NOTE: B rate defined from excited state
        
        if i==excitedLevel-1:
            _k1 += f" - ({_B_rate})"
            
        if i==excitedLevel:
            _k1 += f"{_B_rate}"
        
        # END: Einstein Coefficient B

        # Combining all equations into single array
        _rateEquationsCombined.append(_k1)
        _rateCollection.append(_rateEquationsCombined)
        
    # END: For loop for combining different rates processes
    
    # Computing all rates
    _dR_dt = []

    for rates in _rateCollection:
        _temp = reduce(lambda a, b: a+b, rates)
        _dR_dt.append(_temp)
        
    # Adding rare gas atom attachment and dissociation rates
    _attachmentRate0 = f" - k31_0*{nHe}**2*{_N[0]} + kCID1*{nHe}*{_N_He[0]}*p"
    _attachmentRate1 = f" - k31_1*{nHe}**2*{_N[1]} + kCID1*{nHe}*{_N_He[0]}*(1-p)"
    _attachmentRate2 = f" - k32*{nHe}**2*{_N_He[0]} + kCID2*{nHe}*{_N_He[1]}"
    
    _dR_dt[0] += _attachmentRate0
    _dR_dt[1] += _attachmentRate1
    
    # CDHe:
    _dCDHe_dt = _attachmentRate0 + _attachmentRate1 + _attachmentRate2
    _dR_dt.append(_dCDHe_dt)
    
    # CDHe2
    _dCDHe2_dt = f" - ({_attachmentRate2})"
    _dR_dt.append(_dCDHe2_dt)
    
    for i, eq in enumerate(_dR_dt):
        if i>=totallevel:
            fullRateEquation.append(f"NHe{i} = {eq}")
        else:
            fullRateEquation.append(f"N{i} = {eq}")
            
    return _dR_dt

includeSpontaneousEmissionForAllLevels = False
excitedLevel = 1
fullRateEquation = []
nHe = "nHe"
N = collisionalRateDistribution(1, [1, 2, 3, 4, 0, 0])
fullRateEquation

['N0 =  + q_10*nHe*N1 - q_01*nHe*N0 + q_20*nHe*N2 - q_02*nHe*N0 + q_30*nHe*N3 - q_03*nHe*N0 + A_10*N1 - ( + B_01*N0 - B_10*N1 ) - k31_0*nHe**2*N0 + kCID1*nHe*nHe0*p',
 'N1 =  + q_01*nHe*N0 - q_10*nHe*N1 + q_21*nHe*N2 - q_12*nHe*N1 + q_31*nHe*N3 - q_13*nHe*N1 - A_10*N1 + B_01*N0 - B_10*N1  - k31_1*nHe**2*N1 + kCID1*nHe*nHe0*(1-p)',
 'N2 =  + q_02*nHe*N0 - q_20*nHe*N2 + q_12*nHe*N1 - q_21*nHe*N2 + q_32*nHe*N3 - q_23*nHe*N2',
 'N3 =  + q_03*nHe*N0 - q_30*nHe*N3 + q_13*nHe*N1 - q_31*nHe*N3 + q_23*nHe*N2 - q_32*nHe*N3',
 'NHe4 =  - k31_0*nHe**2*N0 + kCID1*nHe*nHe0*p - k31_1*nHe**2*N1 + kCID1*nHe*nHe0*(1-p) - k32*nHe**2*nHe0 + kCID2*nHe*nHe1',
 'NHe5 =  - ( - k32*nHe**2*nHe0 + kCID2*nHe*nHe1)']

In [20]:
from scipy.constants import m_p
fwhm = lambda freq, temp, m: np.sqrt( (8*Boltzmann*temp*np.log(2)) / (m*m_p*speed_of_light**2) ) * freq

freq = 117.692360 * 1e9
fwhm(freq, 5, 28)*1e-3

35.49229053880689